In [1]:
# use cython compiled modules for faster execution
from scripts.cython.edgelistParser import edgelistParser
from scripts.cython.utils import list2matrix, getInOutDegree, transposeEdgelist, reorderEdgelist
#from scripts.cython.pageRank import approximateSimrank, localPageRank
from scripts.cython.plotNetwork import plotNetworkClusters

from scripts.cpp_libs.pageRankCpp import cppLocalPageRank as localPageRank

import numpy as np
from matplotlib import pyplot as plt

In [11]:
# load the Facebook dataset
A, N, original_idxs = edgelistParser("datasets/facebook/1684.edges", "raw_list")
print(N)

786


In [12]:
%timeit localPageRank(A, c=0.85, return_only_neighbours=True, epsilon=1e-2)

20.4 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
# N=333:  5.77 ms ± 29.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# N=786:  20.4 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
# N=1034: 29.4 ms ± 344 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)